In [1]:
from __future__ import print_function
import keras
#from keras.datasets import cifar10
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np
import os

import h5py
#model = VGG16(weights='imagenet', include_top=False)

import tensorflow as tf
from keras.backend import set_session,tensorflow_backend
config = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True,visible_device_list="0"))
#allow_growth=True: 利用量に合わせてGPUのメモリを動的確保(ちょっとだけ遅くなるけどignorable)
#visible_device_list="0,1,2,3": 利用するGPUのdevice番号を(何故か文字列で)指定する．他の人と被らないようにする．

# config.gpu_options.per_process_gpu_memory_fraction = 0.2 # 上限を0.2に抑える(optional)
set_session(tf.Session(config=config))
#top_model_weights_path = 'fc_model.h5'
batch_size = 32
num_classes = 2
epochs = 10
data_augmentation = False
#num_predictions = 2
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_food_trained_model.h5'


Using TensorFlow backend.
/root/.pyenv/versions/3.6.1/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import cv2
import glob2
import re
import os.path
#import os

def getGTfromPath(path):
    label = os.path.dirname(path).split('/')[-1]
    if label == 'food':
        return [1]
    return [0]
def loadImage(path, target_size=(224,224,3)):
    img = cv2.imread(path)
    return np.resize(img,target_size)

train_filelist = glob2.glob('./data/train/*/*.png')
y_train = [getGTfromPath(p) for p in train_filelist]
x_train = [loadImage(p) for p in train_filelist]
#print(y_train[0:10])

test_filelist = glob2.glob('./data/test/*/*.png')
x_test = [loadImage(p) for p in test_filelist]
y_test = [getGTfromPath(p) for p in test_filelist]

'''
#train_filelist2 = os.listdir('./data/train/nofood/')
print(sorted(train_filelist1)[0])
test_filelist1 = os.listdir('./data/test/food')
test_filelist2 = os.listdir('./data/test/nofood/')

train_imglist1 = []
train_imglist2 = []

test_imglist1 = []
test_imglist2 = []

train_tag1 = []
train_tag2 = []

test_tag1 = []
test_tag2 = []

for train_imgpath0 in train_filelist1:
    train_imgpath1 = os.path.join('./data/train/food/', train_imgpath0)
    train_img1 = cv2.imread(train_imgpath1)
    train_imglist1.append(np.resize(train_img1,(224,224,3)))
#for train_imgpath3 in train_filelist2:
    #train_imgpath2 = os.path.join('./data/train/nofood',train_imgpath3)
    #train_img2 = cv2.imread(train_imgpath2)
    #train_imglist2.append(np.resize(train_img2,(224,224,3)))
    
for test_imgpath0 in test_filelist1:
    test_imgpath1 = os.path.join('./data/train/food/', test_imgpath0)
    test_img1 = cv2.imread(test_imgpath1)
    test_imglist1.append(np.resize(test_img1,(224,224,3)))
#for test_imgpath3 in test_filelist2:
    #test_imgpath2 = os.path.join('./data/train/nofood',test_imgpath3)
    #test_img2 = cv2.imread(test_imgpath2)
    #test_imglist2.append(np.resize(test_img2,(224,224,3)))
    
    
train_tag1 = [1]*len(train_filelist1)#food
train_tag2 = [0]*len(train_filelist2)#nofood

test_tag1 = [1]*len(test_filelist1)#food
test_tag2 = [0]*len(test_filelist2)#nofood

#x_train = [imglist1, imglist2]
train_imglist1 = train_imglist1.extend(train_imglist2)
x_train = train_imglist1
train_tag1.extend(train_tag2)
y_train = train_tag1

test_imglist1 = test_imglist1.extend(test_imglist2)
x_test = test_imglist1
test_tag1.extend(test_tag2)
y_test = test_tag1
'''
#y_train = [train_tag1,train_tag2]
#y_test = [test_tag1,test_tag2]
#print(y_train)
#print(y_test)

#y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

print(np.array(x_train).shape)
print(np.array(y_train).shape)
print(np.array(x_test).shape)
print(np.array(y_test).shape)


(924, 224, 224, 3)
(924, 1)
(236, 224, 224, 3)
(236, 1)


In [ ]:
from keras import applications
from keras import Model
#model = applications.VGG16(weights='imagenet', include_top=False)
head_model = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=None, input_shape=(224,224,3), pooling=None, classes=2)
#model.summary()

# build a classifier model to put on top of the convolutional model
x = Flatten()(head_model.output)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
preds = Dense(1, activation='sigmoid')(x)


model = Model(head_model.input,preds)
#top_model.load_weights(top_model_weights_path)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [ ]:

# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
#model.compile(loss='categorical_crossentropy',
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

x_train = np.array(x_train)
#print(x_train[1].shape)
x_train = x_train.astype('float32')

x_test = np.array(x_test)
x_test = x_test.astype('float32')

x_train /= 255
x_test /= 255

y_train = np.array(y_train)
y_test = np.array(y_test)

#print(np.array(x_train))
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True
             )
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    history = model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=int(np.ceil(x_train.shape[0] / float(batch_size))),
                        epochs=epochs,
                        #validation_data=(x_test, y_test),
                        workers=4)

Not using data augmentation.
Train on 924 samples, validate on 236 samples
Epoch 1/10
924/924 [==============================] - 25s 27ms/step - loss: 0.7082 - acc: 0.5823 - val_loss: 0.6447 - val_acc: 0.6314
Epoch 2/10
896/924 [============================>.] - ETA: 0s - loss: 0.6644 - acc: 0.6328

In [ ]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

#Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])